In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np


def fill_padding(data, max_len, pad_id):
    if len(data) < max_len:
        pad_len = max_len - len(data)
        padding = [pad_id for _ in range(pad_len)]
        # try:
        # print("data: ", data)
        # print("padding: ", padding)
        data = torch.tensor(data + padding)

        # except BaseException:
        #   print("____________________error_________________")
        #   print(data)
        #   print(padding)


    else:
        data = torch.tensor(data[: max_len])
    return data


class articleDataset(Dataset):
    def __init__(self, journal, title, abstruct, label_list, token2id):
        self.journal = journal
        self.title = title
        self.abstruct = abstruct
        self.label_list = label_list
        self.token2id = token2id

    def __len__(self):
        return len(self.journal)

    def __getitem__(self, item):
        return {
            'journal': self.journal[item],
            # 'tokens': fill_padding(self.text_list[item], max_len=self.max_len, pad_id=0),
            'title': fill_padding(self.title[item], max_len=32, pad_id=0),
            'abstruct': fill_padding(self.abstruct[item], max_len=512, pad_id=0),
            'label': torch.tensor(self.label_list[item])
        }

def shuffle_dataset(journal, title, abstruct, label):
    length = len(journal)
    rng = np.random.default_rng(1235)
    index = np.arange(length)
    # print(index)
    rng.shuffle(index)
    # print(index)
    return journal[index], np.array(title)[index], np.array(abstruct)[index], label[index]

def preprocess(file_path):
    df = pd.read_excel(file_path)
    df.replace(np.nan, '[PAD]', inplace=True)
    text_list = []
    title = df['标题']
    abstruct = df['摘要']
    # with open(file_path, 'r', encoding='utf-8') as f:
    #     f.readline()
    for item in title:
        text_list.append(item)
        # print(item)
    for item in abstruct:
        text_list.append(item)
    return text_list


def build_vocab(*text_list):
    token_set = set()
    for idv_list in text_list:
        for tokens in idv_list:
            # print(tokens)
            for i in tokens.split(' '):
                # print(i)
                # print("\n")
                token_set.add(i)

    token2id, id2token = {}, {}
    token2id['[PAD]'] = 0
    id2token[0] = '[PAD]'
    cnt = 1
    for i in token_set:
        token2id[i] = cnt
        id2token[cnt] = i
        cnt += 1

    return token2id, id2token

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn.functional as F

class BiLSTM_Atten(nn.Module):
    def __init__(self, vocab_size):
        super(BiLSTM_Atten, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=128, padding_idx=0)
        self.lstm = nn.LSTM(input_size=128,
                            hidden_size=256,
                            num_layers=2,
                            dropout=0.5,
                            batch_first=True,
                            bidirectional=True)
        self.tanh = nn.Tanh()
        # attention层权重
        self.w = nn.Linear(256 * 2, 256 * 2)
        self.journal_layer = nn.Sequential(
            nn.Linear(16,64),
            nn.ReLU(),
            nn.Linear(64,16),
        )
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(256 + 16, 2)
        # self.liner1 = nn.Linear(512, 256)
        # self.liner2 = nn.Linear(256, 2)

    def forward(self, journal, title, abstruct):
        title = self.embedding(title)
        abstruct = self.embedding(abstruct)
        # print("journal: ", journal.unsqueeze(1).shape)
        # print("title: ", title.shape)
        # print("abstruct: ", abstruct.shape)
        inputs = torch.cat((title, abstruct), 1)
        output, (hn, cn) = self.lstm(inputs)
        # output = pad_packed_sequence(output, batch_first=True, padding_value=0, total_length=64)[0]
        # print(hn[-1, :, :].shape)
        output = self.tanh(output)

        alpha = self.w(output)
        alpha = F.softmax(alpha, dim=1)
        out = output * alpha
        out = torch.sum(out, 1)
        out = self.tanh(out)
        out = self.dropout(out)

        journal_feature = self.journal_layer(journal.float())
        out = self.linear(torch.cat((hn[-1, :, :], journal_feature.to(torch.float32)), 1))
        out = F.softmax(out, dim=1)

        return out



In [ ]:
import torch.nn.functional as F
def focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.7,
    gamma: float = 2,
    reduction: str = "mean",
) -> torch.Tensor:
    """
    Args:
        inputs: A float tensor of arbitrary shape.
                The predictions which have been sigmod for each example.
        targets: A float tensor with the same shape as inputs. Stores the binary
                 classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        alpha: (optional) Weighting factor in range (0,1) to balance
                positive vs negative examples. Default = -1 (no weighting).
        gamma: Exponent of the modulating factor (1 - p_t) to
               balance easy vs hard examples.
        reduction: 'none' | 'mean' | 'sum'
                 'none': No reduction will be applied to the output.
                 'mean': The output will be averaged.
                 'sum': The output will be summed.
    Returns:
        Loss tensor with the reduction option applied.
    """
    inputs = inputs.float()
    targets = targets.float()
    p = inputs
    ce_loss = F.binary_cross_entropy(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

In [ ]:
!pip install thop

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
# from model import TextRNN
# from data_processor import articleDataset, preprocess, build_vocab, shuffle_dataset
import torch.nn as nn
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from thop import profile
import time

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

batch_size = 64
lr = 3e-5
EPOCHS = 100


def evaluate(model, dataloader):
    model.eval()

    pred_tags = []
    true_tags = []
    probs = []
    total_loss = 0.
    # loss_fn = nn.CrossEntropyLoss()
    loss_fn = focal_loss
    with torch.no_grad():
        # i = 0
        for batch in dataloader:
            # print(i)
            # i += 1
            journal = batch['journal'].to(device)
            title = batch['title'].to(device)
            abstruct = batch['abstruct'].to(device)
            label = batch['label'].to(device)

            model.zero_grad()
            # logits = model(journal, title, abstruct)
            # loss = loss_fn(logits, label)
            logits = model(journal, title, abstruct)
            label = label.to(device=device, dtype=torch.float64)
            loss = loss_fn(logits[:, 1].unsqueeze(1), label)
            total_loss += loss.item()

            pred = torch.argmax(logits, dim=1).cpu()
            prob = logits[:, 1].unsqueeze(1).cpu()
            # print(pred.shape)

            pred_tags.extend(pred)
            probs.extend(prob)
            # label = label.to(device=device, dtype=torch.int64)
            # print(label.squeeze(1).shape)
            true_tags.extend(label.squeeze(1).cpu())
            # print(classification_report(true_tags, pred_tags))

    # 计算specificity、sensitivity
    tn, fp, fn, tp = confusion_matrix(true_tags, pred_tags).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)

    assert len(pred_tags) == len(true_tags)

    return {
        'loss': total_loss / len(dataloader),
        'precision': precision_score(true_tags, pred_tags),
        'recall': recall_score(true_tags, pred_tags),
        'f1': f1_score(true_tags, pred_tags),
        'accuracy': accuracy_score(true_tags, pred_tags),
        'classification_report': classification_report(true_tags, pred_tags, digits=4),
        'auc': roc_auc_score(true_tags, probs),
        'specificity': specificity,
        'sensitivity': sensitivity
    }


def main():
    # 建立词汇表
    text_list = preprocess('/content/drive/MyDrive/neckpain_data/新样本.xlsx')
    token2id, id2token = build_vocab(text_list)

    # 建立训练集、测试集
    df = pd.read_excel('/content/drive/MyDrive/neckpain_data/新样本.xlsx')
    # df.replace(np.nan, '[PAD]', inplace=True)
    df_0 = df.loc[df['正样本'] == 0]
    df_1 = df.loc[df['正样本'] == 1]
    title_0 = df_0['标题'].values
    title_1 = df_1['标题'].values
    abstruct_0 = df_0['摘要']
    abstruct_1 = df_1['摘要']
    abstruct_0.replace(np.nan, '[PAD]', inplace=True)
    abstruct_1.replace(np.nan, '[PAD]', inplace=True)
    abstruct_0 = abstruct_0.values
    abstruct_1 = abstruct_1.values
    df_0.replace(np.nan, 0, inplace=True)
    df_1.replace(np.nan, 0, inplace=True)

    X_0 = df_0.drop(columns=['标题', '摘要', '正样本', '关键词', '期刊', '期刊简称', 'ID', 'academy_sciences']).values
    X_1 = df_1.drop(columns=['标题', '摘要', '正样本', '关键词', '期刊', '期刊简称', 'ID', 'academy_sciences']).values
    Y_0 = df_0['正样本'].values
    Y_1 = df_1['正样本'].values

    rs = np.random.RandomState(42)
    L = list(rs.randint(0, len(X_0), int(7 / 3 * len(X_1))))
    X_0 = X_0[L]
    Y_0 = Y_0[L]
    title_0 = title_0[L]
    abstruct_0 = abstruct_0[L]

    journal_train_X_0, journal_test_X_0, train_Y_0, test_Y_0 = train_test_split(X_0, Y_0, train_size=0.80, random_state=42)
    journal_train_X_1, journal_test_X_1, train_Y_1, test_Y_1 = train_test_split(X_1, Y_1, train_size=0.80, random_state=42)
    title_train_X_0, title_test_X_0, _, _ = train_test_split(title_0, Y_0, train_size=0.80, random_state=42)
    title_train_X_1, title_test_X_1, _, _ = train_test_split(title_1, Y_1, train_size=0.80, random_state=42)
    abstruct_train_X_0, abstruct_test_X_0, _, _ = train_test_split(abstruct_0, Y_0, train_size=0.80, random_state=42)
    abstruct_train_X_1, abstruct_test_X_1, _, _ = train_test_split(abstruct_1, Y_1, train_size=0.80, random_state=42)

    # print(title_train_X_1)

    title_train_X_1 = [[token2id[i] for i in text.split(' ')] for text in title_train_X_1]
    title_train_X_0 = [[token2id[i] for i in text.split(' ')] for text in title_train_X_0]
    abstruct_train_X_1 = [[token2id[i] for i in text.split(' ')] for text in abstruct_train_X_1]
    abstruct_train_X_0 = [[token2id[i] for i in text.split(' ')] for text in abstruct_train_X_0]

    title_test_X_1 = [[token2id[i] for i in text.split(' ')] for text in title_test_X_1]
    title_test_X_0 = [[token2id[i] for i in text.split(' ')] for text in title_test_X_0]
    abstruct_test_X_1 = [[token2id[i] for i in text.split(' ')] for text in abstruct_test_X_1]
    abstruct_test_X_0 = [[token2id[i] for i in text.split(' ')] for text in abstruct_test_X_0]

    train_journal = torch.from_numpy(np.vstack((journal_train_X_1.astype(float), journal_train_X_0.astype(float)))).to(device)
    # train_journal = np.vstack((journal_train_X_1, journal_train_X_0))
    train_title = title_train_X_1 + title_train_X_0
    train_abstruct = abstruct_train_X_1 + abstruct_train_X_0
    train_Y = torch.from_numpy(np.vstack((train_Y_1.reshape(len(train_Y_1), 1), train_Y_0.reshape(len(train_Y_0), 1)))).to(device)
    # train_Y = np.vstack((train_Y_1.reshape(len(train_Y_1), 1), train_Y_0.reshape(len(train_Y_0), 1)))
    train_journal, train_title, train_abstruct, train_Y = shuffle_dataset(train_journal, train_title, train_abstruct, train_Y)

    test_journal = torch.from_numpy(np.vstack((journal_test_X_1.astype(float), journal_test_X_0.astype(float)))).to(device)
    # test_journal = np.vstack((journal_test_X_1, journal_test_X_0))
    test_title = title_test_X_1 + title_test_X_0
    test_abstruct = abstruct_test_X_1 + abstruct_test_X_0
    test_Y = torch.from_numpy(np.vstack((test_Y_1.reshape(len(test_Y_1), 1), test_Y_0.reshape(len(test_Y_0), 1)))).to(device)
    # test_Y = np.vstack((test_Y_1.reshape(len(test_Y_1), 1), test_Y_0.reshape(len(test_Y_0), 1)))
    test_journal, test_title, test_abstruct, test_Y = shuffle_dataset(test_journal, test_title, test_abstruct, test_Y)



    train_dataset = articleDataset(train_journal, train_title, train_abstruct, train_Y, token2id)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = articleDataset(test_journal, test_title, test_abstruct, test_Y, token2id)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    model = BiLSTM_Atten(vocab_size=len(token2id)).to(device)
    # for param in model.parameters():
    #   param = 0

    optimizer = AdamW(model.parameters(), lr=lr)
    # loss_fn = nn.CrossEntropyLoss()
    loss_fn = focal_loss

    for epoch in range(EPOCHS):
        model.train()
        start_time = time.time()
        for step, batch in enumerate(train_dataloader):
            journal = batch['journal'].to(device)
            title = batch['title'].to(device)
            abstruct = batch['abstruct'].to(device)
            label = batch['label'].to(device)

            model.zero_grad()

            logits = model(journal, title, abstruct)
            # print(logits[:, 1].unsqueeze(1))
            # print(label)
            label = label.to(device=device, dtype=torch.float64)
            loss = loss_fn(logits[:, 1].unsqueeze(1), label)/batch_size
            # print(loss.item())

            loss.backward()
            optimizer.step()

            if step % (len(train_dataloader) // 9) == 0:
                print("epoch: {} step: {}/{}".format(epoch, step, len(train_dataloader)))

        end_time = time.time()

        # 计算每秒钟的训练样本数（即MLOPS）
        num_samples = len(train_dataset)
        training_time = end_time - start_time
        samples_per_second = num_samples / training_time

        torch.save(model.state_dict(), '/content/drive/MyDrive/neckpain_model/model_last.pt')

        train_metrics = evaluate(model, train_dataloader)
        print("test")
        print("epoch: {} train_loss: {} train_acc: {:.2f}% train_f1: {:.2f}%".format(epoch,
                                                                                     train_metrics['loss'],
                                                                                     train_metrics['accuracy'] * 100,
                                                                                     train_metrics['f1'] * 100))
        # print(train_metrics['classification_report'])
        val_metrics = evaluate(model, val_dataloader)
        print("          val_loss: {} val_acc {:.2f}% val_f1: {:.2f}%".format(val_metrics['loss'],
                                                                               val_metrics['accuracy'] * 100,
                                                                               val_metrics['f1'] * 100))
        print('BiLSTM_Atten AUC = ', val_metrics['auc'])
        print('Specificity = ', val_metrics['specificity'])
        print('Sensitivity = ', val_metrics['sensitivity'])
        # 输出模型信息
        flops, params = profile(model, inputs=(journal, title, abstruct))
        print("Flops: {:.2f}".format(flops))
        print("MLOPS: {:.2f}".format(samples_per_second))
        print(val_metrics['classification_report'])
        # eval(model, test_journal, test_title, test_abstruct, test_Y, token2id)


if __name__ == '__main__':
    if not os.path.exists('/content/drive/MyDrive/neckpain_model'):
        os.mkdir('/content/drive/MyDrive/neckpain_model')

    main()


<ipython-input-7-940f0a9f3f5b>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abstruct_0.replace(np.nan, '[PAD]', inplace=True)
<ipython-input-7-940f0a9f3f5b>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abstruct_1.replace(np.nan, '[PAD]', inplace=True)
<ipython-input-7-940f0a9f3f5b>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0.replace(np.nan, 0, inplace=True)
<ipython-input-7-940f0a9f3f5b>:100: SettingWithCopyWarning

epoch: 0 step: 0/42
epoch: 0 step: 4/42
epoch: 0 step: 8/42
epoch: 0 step: 12/42
epoch: 0 step: 16/42
epoch: 0 step: 20/42
epoch: 0 step: 24/42
epoch: 0 step: 28/42
epoch: 0 step: 32/42
epoch: 0 step: 36/42
epoch: 0 step: 40/42


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)
<ipython-input-2-f0d

test
epoch: 0 train_loss: 0.08230928384831973 train_acc: 55.71% train_f1: 47.36%


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)
<ipython-input-2-f0d

...

epoch: 99 step: 0/42
epoch: 99 step: 4/42
epoch: 99 step: 8/42
epoch: 99 step: 12/42
epoch: 99 step: 16/42
epoch: 99 step: 20/42
epoch: 99 step: 24/42
epoch: 99 step: 28/42
epoch: 99 step: 32/42
epoch: 99 step: 36/42
epoch: 99 step: 40/42


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)
<ipython-input-2-f0d

test
epoch: 99 train_loss: 7.019506655784166e-05 train_acc: 100.00% train_f1: 100.00%


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)


          val_loss: 0.37712549892338837 val_acc 79.55% val_f1: 64.23%
BiLSTM_Atten AUC =  0.8327339846609172
Specificity =  0.8742004264392325
Sensitivity =  0.6119402985074627
[INFO] Register count_lstm() for <class 'torch.nn.modules.rnn.LSTM'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
Flops: 80358815488.00
MLOPS: 182.96
              precision    recall  f1-score   support

         0.0     0.8402    0.8742    0.8568       469
         1.0     0.6758    0.6119    0.6423       201

    accuracy                         0.7955       670
   macro avg     0.7580    0.7431    0.7496       670
weighted avg     0.7909    0.7955    0.7925       670

